In [ ]:
# pose sequence as a NLI premise and label as a hypothesis
import torch
from torch import nn
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# load model pretrained on MNLI
from transformers import BartForSequenceClassification, BartTokenizer
tokenizer = BartTokenizer.from_pretrained('facebook/bart-large-mnli')
sent_model = BartForSequenceClassification.from_pretrained('facebook/bart-large-mnli').to(device)

# add special tokens
tokens = ['chinavirus', 'cherry picker', 'china virus','coronaviruschina', 'ccpvirus', 'covididot'
          'kungflu','chinese virus','wuhanvirus', 'wuhan virus', 'maskless', 'womensuch', 'walkaway',
          'antimask','antivaccine', 'novaccine',  'antivax', 'unvaccinated', 'maskoff', 'boomer', 
          'maskfree', 'babyboomer', 'boomerremover', 'boomer remover', 'wuflu', 'fuckchina', 'NaziRussia', 'SanctionRussiaNow', 'MAGARioters',
          'LockThemUp', 'MAGAMorons', 'Magaidiots']
          
tokenizer.add_tokens(tokens, special_tokens=True)
sent_model.resize_token_embeddings(len(tokenizer))


In [3]:
import pandas as pd
import numpy as np
import json
import random

import torch
from torch import nn
from tqdm import tqdm
from sklearn.metrics import classification_report

In [4]:
# load data
df = pd.read_csv(" ") # path to the data
df.reset_index(inplace=True)

In [5]:
# we use 100 of each category to form a training set
df_train = df.groupby('category').apply(lambda x: x.sample(n=100, random_state=1)).reset_index(drop=True)
# the rest data as test set
df_test = df[~df.index.isin(df_train.index)]

In [6]:
len(df_train), len(df_test)

(600, 3400)

In [7]:
# prepare the training pairs
train = []

for index, row in tqdm(df_train.iterrows()):
    sentence = row['text']
    label = row['ground_truth']
    entailment = 'It is hate speech'
    contradiction = 'It is normal speech'
    pos_tokens = tokenizer.encode(sentence, entailment, max_length=256, truncation=True, return_tensors='pt').to(device)
    neg_tokens = tokenizer.encode(sentence, contradiction, max_length=256, truncation=True, return_tensors='pt').to(device)
    tokens = torch.stack((pos_tokens[0], neg_tokens[0]), dim=0)
    train.append((tokens, label))

600it [00:00, 2161.55it/s]


In [15]:
optimizer = torch.optim.SGD(sent_model.parameters(), lr=0.001, momentum=0.9)
crossentropy = torch.nn.CrossEntropyLoss().to(device)
labels = ['hate', 'normal']

In [14]:
import copy

def train_model(model, dataset, num_epoch=100):
    for epoch in tqdm(range(num_epoch)):
        # print('epoch', epoch)
        loss_epoch = 0
        for posneg, label in tqdm(dataset):
            optimizer.zero_grad()
            output = model(posneg)[0]
            posneg_output = output[:, [0, 2]]
            probs = posneg_output.softmax(dim=1)
            if label == 1.0:
                y = torch.LongTensor([1, 0]).to(device)
            else:
                y = torch.LongTensor([0, 1]).to(device)
            loss = crossentropy(probs, y)

            loss.backward()
            optimizer.step()
            loss_epoch += loss.item()
        loss = loss_epoch / len(dataset)        
        print('epoch', epoch, 'loss', loss)
    
        # return the best model
        if epoch == 0:
            best_loss = loss
            best_model = copy.deepcopy(model)
        else:
            if loss < best_loss:
                best_loss = loss
                best_model = copy.deepcopy(model)
    
    return best_model

In [10]:
# define a prediction function
def predict_normal(tokenizer, model, sentence, labels):
  predit = {}

  premise = sentence
  for label in labels:
    # base 
    hypothesis = f'It is {label} speech.'
    # meta
    # hypothesis = f'This tweet contains {label} speech'
    input_ids = tokenizer.encode(premise, hypothesis, return_tensors='pt').to(device)
    logits = model(input_ids)[0]
    entail_contradiction_logits = logits[:,[0,2]]
    probs = entail_contradiction_logits.softmax(dim=1)
    true_prob = probs[:,1].item() 
    predit[label] = true_prob
  return predit

In [11]:
def calculate(dataframe):
    for cate in dataframe.category.unique().tolist():
        df_sub = dataframe[dataframe['category']==cate]
        print(cate)
        print(classification_report(df_sub['label'], df_sub['prediction'], digits=3))
        print('====================')

In [16]:
# train the model
sent_model.train()
print('=====================\n Begin training:')
model = train_model(sent_model, train, 10)
# torch.save(model.state_dict(), 'fsl_10.pt')

 Begin training:


100%|██████████| 600/600 [07:25<00:00,  1.35it/s]


epoch 0 loss 0.7062429959575335


100%|██████████| 600/600 [07:21<00:00,  1.36it/s]


epoch 1 loss 0.6960038682818412


...

epoch 7 loss 0.6952378838260969


 90%|█████████ | 9/10 [1:05:26<07:13, 433.33s/it]

epoch 8 loss 0.6940576441089312


100%|██████████| 10/10 [1:12:42<00:00, 436.21s/it]

epoch 9 loss 0.6937162359555562


In [25]:
# evaluation of the meta-learner on the EFL dataset
# define a prediction function
def predict_normal(tokenizer, model, sentence, labels):
  predit = {}

  premise = sentence
  for label in labels:
    # base 
    hypothesis = f'It is {label} speech.'
    # meta
    # hypothesis = f'This tweet contains {label} speech'
    input_ids = tokenizer.encode(premise, hypothesis, return_tensors='pt').to(device)
    logits = model(input_ids)[0]
    entail_contradiction_logits = logits[:,[0,2]]
    probs = entail_contradiction_logits.softmax(dim=1)
    true_prob = probs[:,1].item() 
    predit[label] = true_prob
  return predit

def comparison(prediction, label):
  tp = 0
  fp = 0
  tn = 0
  fn = 0

  # get the prediction label
  pred = max(prediction, key=prediction.get)
  if pred == 'hate':
    pred_label = 1
  else:
    pred_label = 0
  # print(pred, d['label'])
  # calculate the tp, fp, tn, fn values
  if pred_label == label:
    if label == 1:
      tp += 1
    else:
      tn += 1
  else:
    if label == 1:
      fn += 1
    else:
      fp += 1
  
  return tp, fp, tn, fn

def get_report(tp, fp, tn, fn):
  accuracy = (tp + tn) / (tp+fp+tn+fn)
  precision = tp / (tp + fp)
  recall = tp / (tp + fn)
  f1 = (precision*recall) /(precision+recall) * 2
  report = {'accuracy': accuracy, 'precision': precision, 'recall': recall, 'f1': f1}
  return report
labels = ['hate', 'normal']

In [28]:
# evaluate for Asian Hate
results = [0,0,0,0]
# random.shuffle(df_test[df_test['category']=='asian'])
for index, row in tqdm(df_test[df_test['category']=='asian'].iterrows(), total=df_test[df_test['category']=='asian'].shape[0]):
  # print(d['tweet'])
  text = str(row['text'])
  predictions = predict_normal(tokenizer, model, text, labels)
  distribution = comparison(predictions, row['ground_truth'])
  for index, integer in enumerate(distribution):
    results[index] += integer

tp, fp, tn, fn = results
print(tp, fp, tn, fn)
get_report(tp, fp, tn, fn)

100%|██████████| 832/832 [04:26<00:00,  3.12it/s]

211 185 202 234


{'accuracy': 0.4963942307692308,
 'precision': 0.5328282828282829,
 'recall': 0.47415730337078654,
 'f1': 0.5017835909631391}

In [30]:
# evaluate for Ageism Hate
results2 = [0,0,0,0]
for index, row in tqdm(df_test[df_test['category']=='ageism'].iterrows(), total=df_test[df_test['category']=='ageism'].shape[0]):
    text = str(row['text'])
    predictions = predict_normal(tokenizer, model, text, labels)
    distribution = comparison(predictions, row['ground_truth'])
    for index, integer in enumerate(distribution):
        results2[index] += integer

tp, fp, tn, fn = results2
print(tp, fp, tn, fn)
get_report(tp, fp, tn, fn)

100%|██████████| 363/363 [01:57<00:00,  3.09it/s]

69 121 109 64


{'accuracy': 0.4903581267217631,
 'precision': 0.3631578947368421,
 'recall': 0.518796992481203,
 'f1': 0.42724458204334365}

In [31]:
# evaluate for Mask Hate
results3 = [0,0,0,0]
for index, row in tqdm(df_test[df_test['category']=='mask'].iterrows(), total=df_test[df_test['category']=='mask'].shape[0]):
    text = str(row['text'])
    predictions = predict_normal(tokenizer, model, text, labels)
    distribution = comparison(predictions, row['ground_truth'])
    for index, integer in enumerate(distribution):
        results3[index] += integer

tp, fp, tn, fn = results3
print(tp, fp, tn, fn)
get_report(tp, fp, tn, fn)

100%|██████████| 497/497 [02:43<00:00,  3.03it/s]

82 161 164 90


{'accuracy': 0.4949698189134809,
 'precision': 0.3374485596707819,
 'recall': 0.47674418604651164,
 'f1': 0.3951807228915663}

In [32]:
# evaluate for Vaccine Hate
results4 = [0,0,0,0]
for index, row in tqdm(df_test[df_test['category']=='vaccine'].iterrows(), total=df_test[df_test['category']=='mask'].shape[0]):
    text = str(row['text'])
    predictions = predict_normal(tokenizer, model, text, labels)
    distribution = comparison(predictions, row['ground_truth'])
    for index, integer in enumerate(distribution):
        results4[index] += integer

tp, fp, tn, fn = results4
print(tp, fp, tn, fn)
get_report(tp, fp, tn, fn)

 81%|████████▏ | 404/497 [02:13<00:30,  3.03it/s]

71 128 129 76


{'accuracy': 0.49504950495049505,
 'precision': 0.35678391959798994,
 'recall': 0.48299319727891155,
 'f1': 0.4104046242774566}

In [33]:
# evaluate for us_capitol Hate
results5 = [0,0,0,0]
for index, row in tqdm(df_test[df_test['category']=='us_capitol'].iterrows(), total=df_test[df_test['category']=='us_capitol'].shape[0]):
    text = str(row['text'])
    predictions = predict_normal(tokenizer, model, text, labels)
    distribution = comparison(predictions, row['ground_truth'])
    for index, integer in enumerate(distribution):
        results5[index] += integer

tp, fp, tn, fn = results5
print(tp, fp, tn, fn)
get_report(tp, fp, tn, fn)

100%|██████████| 704/704 [03:48<00:00,  3.08it/s]

175 184 206 139


{'accuracy': 0.5411931818181818,
 'precision': 0.48746518105849584,
 'recall': 0.5573248407643312,
 'f1': 0.5200594353640416}

In [34]:
# evaluate for rus_ukr Hate
results6 = [0,0,0,0]
for index, row in tqdm(df_test[df_test['category']=='rus_ukr'].iterrows(), total=df_test[df_test['category']=='rus_ukr'].shape[0]):
    text = str(row['text'])
    predictions = predict_normal(tokenizer, model, text, labels)
    distribution = comparison(predictions, row['ground_truth'])
    for index, integer in enumerate(distribution):
        results6[index] += integer

tp, fp, tn, fn = results6
print(tp, fp, tn, fn)
get_report(tp, fp, tn, fn)

100%|██████████| 600/600 [03:31<00:00,  2.84it/s]

115 186 177 122


{'accuracy': 0.4866666666666667,
 'precision': 0.38205980066445183,
 'recall': 0.48523206751054854,
 'f1': 0.42750929368029744}